## Libabries installations & inital set-up

In [24]:
!pip install openai
!pip install tiktoken
!pip install  cohere

In [25]:
import pandas as pd
import string
import openai
import torch
from better_profanity import profanity
from sentence_transformers import SentenceTransformer
from profanity_check import predict, predict_prob
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
import numpy as np
import cohere, tiktoken
from openai import OpenAI
#from google.colab import userdata

pd.set_option('display.max_colwidth', None)

# Path to reddit joke dataset obtained# from dropbox link
file_path = "Data/reddit_joke_dataset.xlsx"
jokes_df = pd.read_excel(file_path)

c:\Users\adity\anaconda3\Lib\site-packages\openpyxl\worksheet\_read_only.py:79: UserWarning: Unknown extension is not supported and will be removed
  for idx, row in parser.parse():


### Importing Dataset

In [26]:
jokes_df = pd.read_excel(file_path)
jokes_df['Joke'] = jokes_df['Setup'] + ' ' + jokes_df['Punchline']
jokes_df

c:\Users\adity\anaconda3\Lib\site-packages\openpyxl\worksheet\_read_only.py:79: UserWarning: Unknown extension is not supported and will be removed
  for idx, row in parser.parse():


,Upvotes,Tags,Setup,Punchline,Joke
0,26603,<nsfw>,Why are women and children evacuated first in a disaster?,So we can think about a solution in silence.,Why are women and children evacuated first in a disaster? So we can think about a solution in silence.
1,26514,<nsfw>,Why do riot police like to get to work early?,To beat the crowd.,Why do riot police like to get to work early? To beat the crowd.
2,19153,<nsfw>,What's the difference between everybody and bullets?,Everybody misses Harambe.,What's the difference between everybody and bullets? Everybody misses Harambe.
3,18000,<nsfw>,Why does Donald Trump take Xanax?,For Hispanic attacks.,Why does Donald Trump take Xanax? For Hispanic attacks.
4,13703,<nsfw>,What's the difference between a hooker and jesus?,The look on their face when you're nailing them.,What's the difference between a hooker and jesus? The look on their face when you're nailing them.
...,...,...,...,...,...
34756,0,NaN,You want to know a ironic statement?,My mom was a cancer and she was killed by a giant crab.,You want to know a ironic statement? My mom was a cancer and she was killed by a giant crab.
34757,0,NaN,You want to see something intense?,Visit a campground.,You want to see something intense? Visit a campground.
34758,0,NaN,You're a Jehova's Witness?,What was Jehova accused of?,You're a Jehova's Witness? What was Jehova accused of?
34759,0,NaN,"You've heard about the Ferguson situation, right?",The issues raised aren't exactly black or white.,"You've heard about the Ferguson situation, right? The issues raised aren't exactly black or white."


In [27]:
jokes_df.head(3)

,Upvotes,Tags,Setup,Punchline,Joke
0,26603,<nsfw>,Why are women and children evacuated first in a disaster?,So we can think about a solution in silence.,Why are women and children evacuated first in a disaster? So we can think about a solution in silence.
1,26514,<nsfw>,Why do riot police like to get to work early?,To beat the crowd.,Why do riot police like to get to work early? To beat the crowd.
2,19153,<nsfw>,What's the difference between everybody and bullets?,Everybody misses Harambe.,What's the difference between everybody and bullets? Everybody misses Harambe.


In [28]:
jokes_df.tail(3)

,Upvotes,Tags,Setup,Punchline,Joke
34758,0,NaN,You're a Jehova's Witness?,What was Jehova accused of?,You're a Jehova's Witness? What was Jehova accused of?
34759,0,NaN,"You've heard about the Ferguson situation, right?",The issues raised aren't exactly black or white.,"You've heard about the Ferguson situation, right? The issues raised aren't exactly black or white."
34760,0,NaN,"Your wife and your lawyer are both drowning, what do you do?",Go to dinner or see a movie?,"Your wife and your lawyer are both drowning, what do you do? Go to dinner or see a movie?"


In [29]:
# Function to check for profanity
def check_profanity(text):
    return predict_prob([text])[0]

In [30]:
jokes_df['Setup-Profanity'] = jokes_df['Setup'].apply(check_profanity)

In [31]:
jokes_df['Punchline-Profanity'] = jokes_df['Punchline'].apply(check_profanity)

In [32]:
jokes_df['Profanity'] = jokes_df['Joke'].apply(check_profanity)

In [33]:
jokes_df['Max-Profanity'] = jokes_df[['Profanity', 'Setup-Profanity', 'Punchline-Profanity']].max(axis=1)
# Create a new column 'Profanity-Tag' to store the tag of the part with the highest profanity score
jokes_df['Profanity-Tag'] = jokes_df.apply(lambda row: 'full' if row['Max-Profanity'] == row['Profanity'] else ('setup' if row['Max-Profanity'] == row['Setup-Profanity'] else 'punchline'), axis=1)

In [34]:
jokes_df['Tags'] = jokes_df.apply(lambda row: '<nsfw>' if row['Profanity'] > 0.5 else row['Tags'], axis=1)

In [35]:

jokes_df.head(40)

,Upvotes,Tags,Setup,Punchline,Joke,Setup-Profanity,Punchline-Profanity,Profanity,Max-Profanity,Profanity-Tag
0,26603,<nsfw>,Why are women and children evacuated first in a disaster?,So we can think about a solution in silence.,Why are women and children evacuated first in a disaster? So we can think about a solution in silence.,0.098884,0.131060,0.172346,0.172346,full
1,26514,<nsfw>,Why do riot police like to get to work early?,To beat the crowd.,Why do riot police like to get to work early? To beat the crowd.,0.011423,0.080620,0.023641,0.080620,punchline
2,19153,<nsfw>,What's the difference between everybody and bullets?,Everybody misses Harambe.,What's the difference between everybody and bullets? Everybody misses Harambe.,0.008741,0.055608,0.018664,0.055608,punchline
3,18000,<nsfw>,Why does Donald Trump take Xanax?,For Hispanic attacks.,Why does Donald Trump take Xanax? For Hispanic attacks.,0.012987,0.024366,0.011493,0.024366,punchline
4,13703,<nsfw>,What's the difference between a hooker and jesus?,The look on their face when you're nailing them.,What's the difference between a hooker and jesus? The look on their face when you're nailing them.,0.041994,0.217764,0.128344,0.217764,punchline
5,12975,<nsfw>,Damn girl are you a Rubik's cube?,"Because fuck you, you stupid piece of shit.","Damn girl are you a Rubik's cube? Because fuck you, you stupid piece of shit.",0.651833,1.000000,1.000000,1.000000,punchline
6,12907,<nsfw>,Why was the baby in Africa crying?,It was having a mid-life crisis.,Why was the baby in Africa crying? It was having a mid-life crisis.,0.138796,0.065953,0.128086,0.138796,setup
7,12733,<nsfw>,Why did the Mexican take Xanax?,For Hispanic attacks.,Why did the Mexican take Xanax? For Hispanic attacks.,0.142002,0.024366,0.064012,0.142002,setup
8,12467,<nsfw>,Why are redneck murders so hard to solve?,There's no dental records and all the DNA matches,Why are redneck murders so hard to solve? There's no dental records and all the DNA matches,0.409749,0.030323,0.196931,0.409749,setup
9,12284,<nsfw>,What does Bill say to Hillary after sex?,Honey I'll be home in 20 minutes.,What does Bill say to Hillary after sex? Honey I'll be home in 20 minutes.,0.315344,0.016360,0.094630,0.315344,setup


In [36]:
jokes_df.tail(40)

,Upvotes,Tags,Setup,Punchline,Joke,Setup-Profanity,Punchline-Profanity,Profanity,Max-Profanity,Profanity-Tag
34721,0,NaN,You know what the problem with discount strippers is?,You can only get 'em half off.,You know what the problem with discount strippers is? You can only get 'em half off.,0.026672,0.103065,0.056450,0.103065,punchline
34722,0,NaN,You know what the say about men with big shoes?,They have big socks.,You know what the say about men with big shoes? They have big socks.,0.029526,0.112346,0.054079,0.112346,punchline
34723,0,<nsfw>,You know what the worst part of vaping is?,Telling your parents you're gay.,You know what the worst part of vaping is? Telling your parents you're gay.,0.606174,0.608469,0.857920,0.857920,full
34724,0,NaN,You know what they call morning wood in Westeros?,They call it an Arthur Dayne- its a Sword of Morning.,You know what they call morning wood in Westeros? They call it an Arthur Dayne- its a Sword of Morning.,0.010275,0.085475,0.029931,0.085475,punchline
34725,0,NaN,You know what they said about Titanic?,It was an Iceberg job.,You know what they said about Titanic? It was an Iceberg job.,0.019135,0.114072,0.035348,0.114072,punchline
34726,0,NaN,You know what they say about Hitler?,He was this tall and he lived over there.,You know what they say about Hitler? He was this tall and he lived over there.,0.376406,0.010128,0.070086,0.376406,setup
34727,0,NaN,You know what they say about living on Navy ships?,It has its ups and downs.,You know what they say about living on Navy ships? It has its ups and downs.,0.016243,0.016434,0.012004,0.016434,punchline
34728,0,NaN,You know what they say about wizards with big hands?,The have big staffs,You know what they say about wizards with big hands? The have big staffs,0.068355,0.073656,0.076360,0.076360,full
34729,0,NaN,You know what Trump and the Patriots have in common?,"Everyone hates them both, but they both win!","You know what Trump and the Patriots have in common? Everyone hates them both, but they both win!",0.003548,0.431622,0.027889,0.431622,punchline
34730,0,NaN,You know what?,It's been a Good Friday.,You know what? It's been a Good Friday.,0.063565,0.021673,0.027502,0.063565,setup


In [37]:
clean_jokes_df = jokes_df[jokes_df['Tags'] != '<nsfw>']
nsfw_jokes_df = jokes_df[jokes_df['Tags'] == '<nsfw>']

# Find duplicate jokes based on the 'Joke' column
duplicates = clean_jokes_df.duplicated(subset='Joke', keep=False)
# Filter the DataFrame to only include duplicates
duplicate_jokes_df = clean_jokes_df[duplicates]
# Optionally, you can drop the duplicates from the original DataFrame
clean_jokes_df = clean_jokes_df.drop_duplicates(subset='Joke', keep=False)

clean_jokes_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 27270 entries, 263 to 34760
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Upvotes              27270 non-null  int64  
 1   Tags                 163 non-null    object 
 2   Setup                27270 non-null  object 
 3   Punchline            27270 non-null  object 
 4   Joke                 27270 non-null  object 
 5   Setup-Profanity      27270 non-null  float64
 6   Punchline-Profanity  27270 non-null  float64
 7   Profanity            27270 non-null  float64
 8   Max-Profanity        27270 non-null  float64
 9   Profanity-Tag        27270 non-null  object 
dtypes: float64(4), int64(1), object(5)
memory usage: 2.3+ MB


In [38]:
import pandas as pd

# Assuming duplicate_jokes_df is your DataFrame and 'column_name' is the name of the column you want to sort by
most_common_value = duplicate_jokes_df['Setup'].value_counts().idxmax()

# Now, you can filter the DataFrame to only include rows with the most common value
most_common_jokes = duplicate_jokes_df[duplicate_jokes_df['Setup'] == most_common_value]

# If you want to sort the DataFrame by the most common value, you can do so like this:
most_common_jokes = duplicate_jokes_df.sort_values(by='Setup', ascending=False)

# To sort by the most common value and then by another column (e.g., 'date' in ascending order), you can do:
most_common_jokes = duplicate_jokes_df.sort_values(by=['Setup', 'Punchline'], ascending=[False, False])
most_common_jokes.to_csv("common.csv")

In [39]:
clean_jokes_df.to_csv("clean.csv")

In [40]:
# Load a pre-trained model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Combine 'Setup' and 'Punchline' into a single column for vectorization
#jokes_df['Joke'] = jokes_df['Setup'] + ' ' + jokes_df['Punchline']

# Vectorize the jokes
clean_jokes_df['Embedding'] = clean_jokes_df['Joke'].apply(lambda x: model.encode(x))

In [41]:
# Convert the embeddings column to a 2D numpy array
embeddings_array = np.array(clean_jokes_df['Embedding'].tolist())
# Compute the cosine similarity matrix
cosine_sim_matrix = cosine_similarity(embeddings_array)

In [42]:
# Set a threshold for similarity (e.g., 0.8)
similarity_threshold = 0.8

# Find pairs of jokes that are similar based on the threshold
similar_pairs = np.where(cosine_sim_matrix > similarity_threshold)

# Process the pairs to find duplicates
# This step depends on how you define a "duplicate" in terms of similarity
# For example, you might consider jokes to be duplicates if they have a high similarity score with any other joke


In [43]:
# Assuming similar_pairs is a tuple of two arrays, where the first array contains the indices of the original jokes
# and the second array contains the indices of the similar jokes.

# Step 1: Prepare a dictionary to store similar jokes
similar_jokes_dict = {}

# Step 2: Prepare a set to keep track of used indexes
used_indexes = set()

# Step 3: Populate the dictionary
for original_joke_index, similar_joke_index in zip(*similar_pairs):
    # If the original joke index is not already in the dictionary, add it with an empty list
    if original_joke_index not in similar_jokes_dict:
        similar_jokes_dict[original_joke_index] = []
    
    # Add the similar joke index to the list of similar jokes for the original joke
    # Only add if the similar joke index has not been used before
    if similar_joke_index not in used_indexes:
        similar_jokes_dict[original_joke_index].append(similar_joke_index)
        used_indexes.add(similar_joke_index)

# Step 4: Filter out the original joke index from its own list of similar jokes
for original_joke_index, similar_joke_indexes in similar_jokes_dict.items():
    # Filter out the original joke index from the list of similar jokes
    filtered_similar_joke_indexes = [joke_index for joke_index in similar_joke_indexes if joke_index != original_joke_index]
    
    # Update the list of similar jokes for the current original joke index
    similar_jokes_dict[original_joke_index] = filtered_similar_joke_indexes

# Step 5: Create a new DataFrame
similar_jokes_df = pd.DataFrame(list(similar_jokes_dict.items()), columns=['Original Joke Index', 'Similar Joke Indexes'])

# Now, similar_jokes_df contains the indexes of jokes grouped together based on their similarity,
# with the original joke index not appearing in its own list of similar jokes, and each index is assigned to only one group.



In [44]:
similar_jokes_df

,Original Joke Index,Similar Joke Indexes
0,0,[]
1,1,"[1259, 3220]"
2,2,"[2022, 2096, 2152, 2621, 3451, 3581, 3745, 3746, 3827, 5315, 6248, 7224, 7635, 7636, 7637, 7638, 8104, 9485, 9636, 10375, 10388, 10389, 10390, 11562, 11563, 13037, 13038, 15119, 15120, 15121, 16249, 25014, 25015, 25017]"
3,3,[]
4,4,"[207, 252, 463, 1004, 3003, 3319, 6291, 15245]"
...,...,...
27265,27265,[]
27266,27266,[]
27267,27267,[]
27268,27268,[]


In [45]:
# Assuming similar_jokes_df and clean_jokes_df are already loaded as pandas DataFrames

# Step 2: Extract all similar joke indexes
similar_indexes = []
for index, row in similar_jokes_df.iterrows():
    similar_indexes.extend(row['Similar Joke Indexes'])

# Ensure all indexes are unique
similar_indexes = list(set(similar_indexes))

# Step 3: Filter cleaned_jokes_df to remove similar jokes
filtered_df = clean_jokes_df[~clean_jokes_df.index.isin(similar_indexes)]

# The filtered_df now contains only the original jokes without the similar ones


In [46]:
file_path = "filtered_jokes.csv"
# Read the CSV file
filtered_df = pd.read_csv(file_path)

In [47]:
filtered_df = filtered_df.drop(columns=['Profanity', 'Unnamed: 6'])
filtered_df

,Upvotes,Tags,Setup,Punchline,Joke
0,183,NaN,Why did it take so long to see a picture of Saint West?,Because he was a Tidal exclusive.,Why did it take so long to see a picture of Saint West? Because he was a Tidal exclusive.
1,13,NaN,Who is the original Kardashian?,Saint Christopher,Who is the original Kardashian? Saint Christopher
2,2,<nsfw>,Why did the gynecologist need glasses?,Everything he saw was fuzzy.,Why did the gynecologist need glasses? Everything he saw was fuzzy.
3,0,<nsfw>,Why is 'secret' such a popular password on Brazzers?,You can type it with one hand.,Why is 'secret' such a popular password on Brazzers? You can type it with one hand.
4,9,<nsfw>,What do Japanese emperors and rappers have in common?,They're both wondering where their ninjas are at.,What do Japanese emperors and rappers have in common? They're both wondering where their ninjas are at.
...,...,...,...,...,...
19526,7,<nsfw>,Why does the pedophile like Halloween?,Free shipping.,Why does the pedophile like Halloween? Free shipping.
19527,2,<nsfw>,What's the difference between hardware and software?,A floppy dick,What's the difference between hardware and software? A floppy dick
19528,0,<nsfw>,What's the mating call of a sorority girl?,O My GOd! I am so drunk.,What's the mating call of a sorority girl? O My GOd! I am so drunk.
19529,0,<nsfw>,When did the man know it was time to dump his fat GF?,When he realized it just wasn't going to work out.,When did the man know it was time to dump his fat GF? When he realized it just wasn't going to work out.


In [48]:
filtered_df['Setup-Profanity'] = filtered_df['Setup'].apply(check_profanity)
filtered_df['Punchline-Profanity'] = filtered_df['Punchline'].apply(check_profanity)
filtered_df['Profanity'] = filtered_df['Joke'].apply(check_profanity)


In [49]:
filtered_df['Max-Profanity'] = filtered_df[['Profanity', 'Setup-Profanity', 'Punchline-Profanity']].max(axis=1)

# Create a new column 'Profanity-Tag' to store the tag of the part with the highest profanity score
filtered_df['Profanity-Tag'] = filtered_df.apply(lambda row: 'full' if row['Max-Profanity'] == row['Profanity'] else ('setup' if row['Max-Profanity'] == row['Setup-Profanity'] else 'punchline'), axis=1)
filtered_df['Tags'] = filtered_df.apply(lambda row: '<nsfw>' if row['Max-Profanity'] >= 0.4 else row['Tags'], axis=1)

filtered_df

,Upvotes,Tags,Setup,Punchline,Joke,Setup-Profanity,Punchline-Profanity,Profanity,Max-Profanity,Profanity-Tag
0,183,NaN,Why did it take so long to see a picture of Saint West?,Because he was a Tidal exclusive.,Why did it take so long to see a picture of Saint West? Because he was a Tidal exclusive.,0.000275,0.014543,0.000325,0.014543,punchline
1,13,NaN,Who is the original Kardashian?,Saint Christopher,Who is the original Kardashian? Saint Christopher,0.016527,0.000436,0.000451,0.016527,setup
2,2,<nsfw>,Why did the gynecologist need glasses?,Everything he saw was fuzzy.,Why did the gynecologist need glasses? Everything he saw was fuzzy.,0.001411,0.002528,0.000548,0.002528,punchline
3,0,<nsfw>,Why is 'secret' such a popular password on Brazzers?,You can type it with one hand.,Why is 'secret' such a popular password on Brazzers? You can type it with one hand.,0.000529,0.012441,0.000562,0.012441,punchline
4,9,<nsfw>,What do Japanese emperors and rappers have in common?,They're both wondering where their ninjas are at.,What do Japanese emperors and rappers have in common? They're both wondering where their ninjas are at.,0.001194,0.004426,0.000662,0.004426,punchline
...,...,...,...,...,...,...,...,...,...,...
19526,7,<nsfw>,Why does the pedophile like Halloween?,Free shipping.,Why does the pedophile like Halloween? Free shipping.,0.579848,0.078546,0.498856,0.579848,setup
19527,2,<nsfw>,What's the difference between hardware and software?,A floppy dick,What's the difference between hardware and software? A floppy dick,0.003599,0.999900,0.498887,0.999900,punchline
19528,0,<nsfw>,What's the mating call of a sorority girl?,O My GOd! I am so drunk.,What's the mating call of a sorority girl? O My GOd! I am so drunk.,0.139347,0.431150,0.499275,0.499275,full
19529,0,<nsfw>,When did the man know it was time to dump his fat GF?,When he realized it just wasn't going to work out.,When did the man know it was time to dump his fat GF? When he realized it just wasn't going to work out.,0.582866,0.064764,0.499514,0.582866,setup


In [50]:
clean_jokes_df = filtered_df[filtered_df['Tags'] != '<nsfw>']
nsfw_jokes_df = filtered_df[filtered_df['Tags'] == '<nsfw>']

# Find duplicate jokes based on the 'Joke' column
duplicates = clean_jokes_df.duplicated(subset='Joke', keep=False)
# Filter the DataFrame to only include duplicates
duplicate_jokes_df = clean_jokes_df[duplicates]
# Optionally, you can drop the duplicates from the original DataFrame
clean_jokes_df = clean_jokes_df.drop_duplicates(subset='Joke', keep=False)

clean_jokes_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 16905 entries, 0 to 18758
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Upvotes              16905 non-null  int64  
 1   Tags                 141 non-null    object 
 2   Setup                16905 non-null  object 
 3   Punchline            16905 non-null  object 
 4   Joke                 16905 non-null  object 
 5   Setup-Profanity      16905 non-null  float64
 6   Punchline-Profanity  16905 non-null  float64
 7   Profanity            16905 non-null  float64
 8   Max-Profanity        16905 non-null  float64
 9   Profanity-Tag        16905 non-null  object 
dtypes: float64(4), int64(1), object(5)
memory usage: 1.4+ MB


In [51]:
clean_jokes_df

,Upvotes,Tags,Setup,Punchline,Joke,Setup-Profanity,Punchline-Profanity,Profanity,Max-Profanity,Profanity-Tag
0,183,NaN,Why did it take so long to see a picture of Saint West?,Because he was a Tidal exclusive.,Why did it take so long to see a picture of Saint West? Because he was a Tidal exclusive.,0.000275,0.014543,0.000325,0.014543,punchline
1,13,NaN,Who is the original Kardashian?,Saint Christopher,Who is the original Kardashian? Saint Christopher,0.016527,0.000436,0.000451,0.016527,setup
5,4,NaN,What is the most overused force in science?,Van der Waals.,What is the most overused force in science? Van der Waals.,0.002765,0.001675,0.000674,0.002765,setup
6,9,NaN,Why did the UN outlaw diet pills?,Because they're weapons of mass destruction.,Why did the UN outlaw diet pills? Because they're weapons of mass destruction.,0.003890,0.001299,0.000695,0.003890,setup
7,194,NaN,What did the doctor say to the speeding commuter?,Thank you for your patients.,What did the doctor say to the speeding commuter? Thank you for your patients.,0.007122,0.000680,0.000700,0.007122,setup
...,...,...,...,...,...,...,...,...,...,...
18752,7,NaN,What does Tony Romo and a porn star have in common?,They both know how to choke.,What does Tony Romo and a porn star have in common? They both know how to choke.,0.351525,0.117008,0.398032,0.398032,full
18753,0,NaN,What brand of butter do frogs eat?,Country Croak.,What brand of butter do frogs eat? Country Croak.,0.320881,0.140975,0.398398,0.398398,full
18754,0,NaN,Why doesn't God want to argue with Satan?,Because Satan has more politicians to help him.,Why doesn't God want to argue with Satan? Because Satan has more politicians to help him.,0.362966,0.257964,0.398457,0.398457,full
18755,0,NaN,What do you call it when Drake suicides?,"Jumpman, jumpman, jumpman, jumpman","What do you call it when Drake suicides? Jumpman, jumpman, jumpman, jumpman",0.398561,0.036400,0.398561,0.398561,full


In [52]:
blacklist = pd.read_csv("Terms-to-Block.csv")
#blacklist #= blacklist.replace(',', '', regex=True)
#blacklist.to_csv("Terms-to-Block.csv")

In [53]:
# Assuming 'blacklist_df' is your DataFrame containing terms to block
# and 'clean_jokes_df' is your main DataFrame

# Convert the 'Terms' column of 'blacklist_df' to a list
blacklist_terms = blacklist['Terms'].tolist()

# Function to check if any term in the list appears in the text
def contains_blocked_term(text):
    return any(term in text for term in blacklist_terms)

# Filter 'clean_jokes_df' to remove rows containing any of the blocked terms
filtered_clean_jokes_df = clean_jokes_df[~clean_jokes_df['Joke'].apply(contains_blocked_term)]

# Now, 'filtered_clean_jokes_df' contains only the rows that do not contain any of the blocked terms


In [54]:
filtered_clean_jokes_df

,Upvotes,Tags,Setup,Punchline,Joke,Setup-Profanity,Punchline-Profanity,Profanity,Max-Profanity,Profanity-Tag
0,183,NaN,Why did it take so long to see a picture of Saint West?,Because he was a Tidal exclusive.,Why did it take so long to see a picture of Saint West? Because he was a Tidal exclusive.,0.000275,0.014543,0.000325,0.014543,punchline
1,13,NaN,Who is the original Kardashian?,Saint Christopher,Who is the original Kardashian? Saint Christopher,0.016527,0.000436,0.000451,0.016527,setup
5,4,NaN,What is the most overused force in science?,Van der Waals.,What is the most overused force in science? Van der Waals.,0.002765,0.001675,0.000674,0.002765,setup
8,16,NaN,Why are Austalian grocery stores the best?,Because of their Koala Tea,Why are Austalian grocery stores the best? Because of their Koala Tea,0.000412,0.022087,0.000728,0.022087,punchline
10,4,NaN,What's the best airline to fly around the Seven Kingdoms ?,Southwesteros,What's the best airline to fly around the Seven Kingdoms ? Southwesteros,0.000772,0.036400,0.000772,0.036400,punchline
...,...,...,...,...,...,...,...,...,...,...
18741,147,NaN,Why are there no female necrophiliacs?,Because dead guys can't spend money.,Why are there no female necrophiliacs? Because dead guys can't spend money.,0.128858,0.325976,0.396103,0.396103,full
18750,3,NaN,What do you call a Peruvian Immigrant?,A Lima Beaner,What do you call a Peruvian Immigrant? A Lima Beaner,0.277124,0.213083,0.397611,0.397611,full
18751,1,NaN,Did you know that 95% of Jews aren't Jews?,They're dead,Did you know that 95% of Jews aren't Jews? They're dead,0.276593,0.233661,0.397615,0.397615,full
18754,0,NaN,Why doesn't God want to argue with Satan?,Because Satan has more politicians to help him.,Why doesn't God want to argue with Satan? Because Satan has more politicians to help him.,0.362966,0.257964,0.398457,0.398457,full


In [55]:
filtered_clean_jokes_df.to_csv("final_dataset2.xlsx")

In [56]:
df = pd.read_csv("final_dataset2.xlsx")

In [57]:
words_to_check = ['feminist', 'lesbian', 'gay', 'porn', 'Indian', 'Chinese', 'asian','hooker', '9/11', 'Jew', 'Jews', 'Muslim', 'Hindu', 'ISIS', 'terrorist', 'terrorism', 'transgender', 'lgbt', 'pedo', 'Pedophile', 'suicide', 'redneck', 'allah', 'pms', 'hitler', 'nazi']
pattern = '|'.join(f'.*{word}.*' for word in words_to_check)
df = df[~df['Joke'].str.contains(pattern, case=False, na=False)]
df

,Unnamed: 0,Upvotes,Tags,Setup,Punchline,Joke,Setup-Profanity,Punchline-Profanity,Profanity,Max-Profanity,Profanity-Tag
0,0,183,NaN,Why did it take so long to see a picture of Saint West?,Because he was a Tidal exclusive.,Why did it take so long to see a picture of Saint West? Because he was a Tidal exclusive.,0.000275,0.014543,0.000325,0.014543,punchline
1,1,13,NaN,Who is the original Kardashian?,Saint Christopher,Who is the original Kardashian? Saint Christopher,0.016527,0.000436,0.000451,0.016527,setup
2,5,4,NaN,What is the most overused force in science?,Van der Waals.,What is the most overused force in science? Van der Waals.,0.002765,0.001675,0.000674,0.002765,setup
3,8,16,NaN,Why are Austalian grocery stores the best?,Because of their Koala Tea,Why are Austalian grocery stores the best? Because of their Koala Tea,0.000412,0.022087,0.000728,0.022087,punchline
4,10,4,NaN,What's the best airline to fly around the Seven Kingdoms ?,Southwesteros,What's the best airline to fly around the Seven Kingdoms ? Southwesteros,0.000772,0.036400,0.000772,0.036400,punchline
...,...,...,...,...,...,...,...,...,...,...,...
12261,18735,0,NaN,What does a person that's been laughing for ten minutes say?,OMG HAHAHA I CANNES'T EVEN,What does a person that's been laughing for ten minutes say? OMG HAHAHA I CANNES'T EVEN,0.268857,0.180410,0.395255,0.395255,full
12262,18741,147,NaN,Why are there no female necrophiliacs?,Because dead guys can't spend money.,Why are there no female necrophiliacs? Because dead guys can't spend money.,0.128858,0.325976,0.396103,0.396103,full
12263,18750,3,NaN,What do you call a Peruvian Immigrant?,A Lima Beaner,What do you call a Peruvian Immigrant? A Lima Beaner,0.277124,0.213083,0.397611,0.397611,full
12265,18754,0,NaN,Why doesn't God want to argue with Satan?,Because Satan has more politicians to help him.,Why doesn't God want to argue with Satan? Because Satan has more politicians to help him.,0.362966,0.257964,0.398457,0.398457,full


In [58]:
df = df[~df['Tags'].str.contains('<nsfw>', na=False)]
df
#7605 rows × 6 columns

,Unnamed: 0,Upvotes,Tags,Setup,Punchline,Joke,Setup-Profanity,Punchline-Profanity,Profanity,Max-Profanity,Profanity-Tag
0,0,183,NaN,Why did it take so long to see a picture of Saint West?,Because he was a Tidal exclusive.,Why did it take so long to see a picture of Saint West? Because he was a Tidal exclusive.,0.000275,0.014543,0.000325,0.014543,punchline
1,1,13,NaN,Who is the original Kardashian?,Saint Christopher,Who is the original Kardashian? Saint Christopher,0.016527,0.000436,0.000451,0.016527,setup
2,5,4,NaN,What is the most overused force in science?,Van der Waals.,What is the most overused force in science? Van der Waals.,0.002765,0.001675,0.000674,0.002765,setup
3,8,16,NaN,Why are Austalian grocery stores the best?,Because of their Koala Tea,Why are Austalian grocery stores the best? Because of their Koala Tea,0.000412,0.022087,0.000728,0.022087,punchline
4,10,4,NaN,What's the best airline to fly around the Seven Kingdoms ?,Southwesteros,What's the best airline to fly around the Seven Kingdoms ? Southwesteros,0.000772,0.036400,0.000772,0.036400,punchline
...,...,...,...,...,...,...,...,...,...,...,...
12261,18735,0,NaN,What does a person that's been laughing for ten minutes say?,OMG HAHAHA I CANNES'T EVEN,What does a person that's been laughing for ten minutes say? OMG HAHAHA I CANNES'T EVEN,0.268857,0.180410,0.395255,0.395255,full
12262,18741,147,NaN,Why are there no female necrophiliacs?,Because dead guys can't spend money.,Why are there no female necrophiliacs? Because dead guys can't spend money.,0.128858,0.325976,0.396103,0.396103,full
12263,18750,3,NaN,What do you call a Peruvian Immigrant?,A Lima Beaner,What do you call a Peruvian Immigrant? A Lima Beaner,0.277124,0.213083,0.397611,0.397611,full
12265,18754,0,NaN,Why doesn't God want to argue with Satan?,Because Satan has more politicians to help him.,Why doesn't God want to argue with Satan? Because Satan has more politicians to help him.,0.362966,0.257964,0.398457,0.398457,full


In [59]:
df['Profanity'].max()

0.3995397885156809

In [60]:
df2 = df.drop(columns=['Setup-Profanity', 'Punchline-Profanity', 'Profanity', 'Max-Profanity'])
df2['Profanity'] = df2['Joke'].apply(check_profanity)
df2['Setup-Profanity'] = df2['Setup'].apply(check_profanity)
df2['Punchline-Profanity'] = df2['Punchline'].apply(check_profanity)
df2['Max-Profanity'] = df2[['Profanity', 'Setup-Profanity', 'Punchline-Profanity']].max(axis=1)

# Create a new column 'Profanity-Tag' to store the tag of the part with the highest profanity score
df2['Profanity-Tag'] = df2.apply(lambda row: 'full' if row['Max-Profanity'] == row['Profanity'] else ('setup' if row['Max-Profanity'] == row['Setup-Profanity'] else 'punchline'), axis=1)
df2['Tags'] = df2.apply(lambda row: '<nsfw>' if row['Profanity'] > 0.4 else row['Tags'], axis=1)
df2

,Unnamed: 0,Upvotes,Tags,Setup,Punchline,Joke,Profanity-Tag,Profanity,Setup-Profanity,Punchline-Profanity,Max-Profanity
0,0,183,NaN,Why did it take so long to see a picture of Saint West?,Because he was a Tidal exclusive.,Why did it take so long to see a picture of Saint West? Because he was a Tidal exclusive.,punchline,0.000325,0.000275,0.014543,0.014543
1,1,13,NaN,Who is the original Kardashian?,Saint Christopher,Who is the original Kardashian? Saint Christopher,setup,0.000451,0.016527,0.000436,0.016527
2,5,4,NaN,What is the most overused force in science?,Van der Waals.,What is the most overused force in science? Van der Waals.,setup,0.000674,0.002765,0.001675,0.002765
3,8,16,NaN,Why are Austalian grocery stores the best?,Because of their Koala Tea,Why are Austalian grocery stores the best? Because of their Koala Tea,punchline,0.000728,0.000412,0.022087,0.022087
4,10,4,NaN,What's the best airline to fly around the Seven Kingdoms ?,Southwesteros,What's the best airline to fly around the Seven Kingdoms ? Southwesteros,punchline,0.000772,0.000772,0.036400,0.036400
...,...,...,...,...,...,...,...,...,...,...,...
12261,18735,0,NaN,What does a person that's been laughing for ten minutes say?,OMG HAHAHA I CANNES'T EVEN,What does a person that's been laughing for ten minutes say? OMG HAHAHA I CANNES'T EVEN,full,0.395255,0.268857,0.180410,0.395255
12262,18741,147,NaN,Why are there no female necrophiliacs?,Because dead guys can't spend money.,Why are there no female necrophiliacs? Because dead guys can't spend money.,full,0.396103,0.128858,0.325976,0.396103
12263,18750,3,NaN,What do you call a Peruvian Immigrant?,A Lima Beaner,What do you call a Peruvian Immigrant? A Lima Beaner,full,0.397611,0.277124,0.213083,0.397611
12265,18754,0,NaN,Why doesn't God want to argue with Satan?,Because Satan has more politicians to help him.,Why doesn't God want to argue with Satan? Because Satan has more politicians to help him.,full,0.398457,0.362966,0.257964,0.398457


In [61]:
final = df2.drop(columns=['Setup-Profanity', 'Punchline-Profanity', 'Profanity', 'Max-Profanity', 'Profanity-Tag'])
final

,Unnamed: 0,Upvotes,Tags,Setup,Punchline,Joke
0,0,183,NaN,Why did it take so long to see a picture of Saint West?,Because he was a Tidal exclusive.,Why did it take so long to see a picture of Saint West? Because he was a Tidal exclusive.
1,1,13,NaN,Who is the original Kardashian?,Saint Christopher,Who is the original Kardashian? Saint Christopher
2,5,4,NaN,What is the most overused force in science?,Van der Waals.,What is the most overused force in science? Van der Waals.
3,8,16,NaN,Why are Austalian grocery stores the best?,Because of their Koala Tea,Why are Austalian grocery stores the best? Because of their Koala Tea
4,10,4,NaN,What's the best airline to fly around the Seven Kingdoms ?,Southwesteros,What's the best airline to fly around the Seven Kingdoms ? Southwesteros
...,...,...,...,...,...,...
12261,18735,0,NaN,What does a person that's been laughing for ten minutes say?,OMG HAHAHA I CANNES'T EVEN,What does a person that's been laughing for ten minutes say? OMG HAHAHA I CANNES'T EVEN
12262,18741,147,NaN,Why are there no female necrophiliacs?,Because dead guys can't spend money.,Why are there no female necrophiliacs? Because dead guys can't spend money.
12263,18750,3,NaN,What do you call a Peruvian Immigrant?,A Lima Beaner,What do you call a Peruvian Immigrant? A Lima Beaner
12265,18754,0,NaN,Why doesn't God want to argue with Satan?,Because Satan has more politicians to help him.,Why doesn't God want to argue with Satan? Because Satan has more politicians to help him.


In [62]:
final = final[~final['Tags'].str.contains('<nsfw>', na=False)]
final

,Unnamed: 0,Upvotes,Tags,Setup,Punchline,Joke
0,0,183,NaN,Why did it take so long to see a picture of Saint West?,Because he was a Tidal exclusive.,Why did it take so long to see a picture of Saint West? Because he was a Tidal exclusive.
1,1,13,NaN,Who is the original Kardashian?,Saint Christopher,Who is the original Kardashian? Saint Christopher
2,5,4,NaN,What is the most overused force in science?,Van der Waals.,What is the most overused force in science? Van der Waals.
3,8,16,NaN,Why are Austalian grocery stores the best?,Because of their Koala Tea,Why are Austalian grocery stores the best? Because of their Koala Tea
4,10,4,NaN,What's the best airline to fly around the Seven Kingdoms ?,Southwesteros,What's the best airline to fly around the Seven Kingdoms ? Southwesteros
...,...,...,...,...,...,...
12261,18735,0,NaN,What does a person that's been laughing for ten minutes say?,OMG HAHAHA I CANNES'T EVEN,What does a person that's been laughing for ten minutes say? OMG HAHAHA I CANNES'T EVEN
12262,18741,147,NaN,Why are there no female necrophiliacs?,Because dead guys can't spend money.,Why are there no female necrophiliacs? Because dead guys can't spend money.
12263,18750,3,NaN,What do you call a Peruvian Immigrant?,A Lima Beaner,What do you call a Peruvian Immigrant? A Lima Beaner
12265,18754,0,NaN,Why doesn't God want to argue with Satan?,Because Satan has more politicians to help him.,Why doesn't God want to argue with Satan? Because Satan has more politicians to help him.


In [63]:
import json

# Function to write data in JSONL format
def write_jsonl(data, filename):
    with open(filename, 'w') as f:
        for item in data:
            f.write(json.dumps(item) + '\n')

# Convert the DataFrame to a list of dictionaries
data_list = final.to_dict('records')

# Calculate the split index
split_index = int(len(data_list) * 0.7)

# Split the data into training and testing sets
train_data = data_list[:split_index]
test_data = data_list[split_index:]

# Function to format each item for the desired JSONL structure
def format_item_for_jsonl(item):
    return {
        "messages": [
            {"role": "system", "content": "Goofy is a joke writer for a standup comedian."},
            {"role": "user", "content": f"setup: {item['Setup']}"},
            {"role": "assistant", "content": f"punchline: {item['Punchline']}"}
        ]
    }

# Apply the formatting to each item in the training and testing sets
train_data = [format_item_for_jsonl(item) for item in train_data]
test_data = [format_item_for_jsonl(item) for item in test_data]

# Write the training data to a JSONL file
write_jsonl(train_data, 'LFT_training.jsonl')

# Write the testing data to a JSONL file
write_jsonl(test_data, 'LFT_validation.jsonl')

print("Data has been split and written to 'fine_tuning_J3F_training.jsonl' and 'ft_gen_J10_validation.jsonl'")

Data has been split and written to 'fine_tuning_J3F_training.jsonl' and 'ft_gen_J10_validation.jsonl'
